# AI Project: Employee Attrition Prediction - **source code**

## Table of Contents
- [Framework](#framework)
- [Initial dataset](#initial-dataset)
- [Ethic](#ethic)
- [Data wrangling](#data-wrangling)
- [Model](#model)
  - [Implementation](#implementation)
  - [Quality indicators](#quality-indicators)
  - [Graph and observation](#graph-and-observation)


## Framework

In [6]:
import pandas as pd
import numpy as np
# ...

## Initial dataset

little description of the data set

## Ethic

blabla ethic
What we delete

## Data wrangling

In [ ]:
# Load data
general_df = pd.read_csv('general_data.csv')
manager_survey_df = pd.read_csv('manager_survey_data.csv')
employee_survey_df = pd.read_csv('employee_survey_data.csv')
in_time = pd.read_csv(r"D:\AI\prj\in_out_time\in_time.csv")
out_time = pd.read_csv(r"D:\AI\prj\in_out_time\out_time.csv")

## model

### Implementation

In [8]:
# implemente the model
# with  pipeline

### Quality indicators

In [9]:
# MSE, MAE RMSE, R²
# Cross validation

### Graph and observation

In [10]:
# plt if we have